In [75]:

class HMM():

    '''
    class that represents a Hidden Markov Model (HMM)
    contains methods that analyse aspects of the HMM
    '''
    def __init__(self,transMatrix,emitMatrix,emissions):
        '''

        :param transMatrix: Nested dictionary representing a transition matrix
        key:value --> state: RowDictionary
        Where RowDictionary is a dictionary that represents a single row in the matrix.
        key:value --> state: transition pr

        :param emitMatrix: Nested dictionary representing an emission matrix
        key:value --> state: RowDictionary
        Where RowDictionary is a dictionary that represents a single row in the matrix.
        key:value --> state: emission pr

        :param emissions: String representing the emissions from the HMM
        '''

        self.transMatrix = transMatrix
        self.emitMatrix = emitMatrix
        self.emissions = emissions

    def genGraph(self):
        '''

        :return: list of dictionaries that represents an HMM
        '''
        graph = []
        for i in self.emissions:
            colDict = {}
            for state in self.transMatrix.keys():
                colDict[state] = 0
            graph.append(colDict)

        return graph
    def backward(self):
        '''
        Similar to forward,except it starts on the opposite end of the graph
        Runs genGraph() to crate an empty graph and then scores it from the sink to the source.
        The score of each node is equal to the sum of the weights of each edge that goes into it.
        Weight = score of previous node * transition pr * emission pr
        :return: list of dictionaries, each dictionary represents the ith layer in an HMM
                    key:value = state:score
        '''
        graph = self.genGraph()
        states = self.transMatrix.keys()

        # score the rest of the graph
        for state in states:
            graph[0][state] = 1

        #score the rest of the graph
        for i in range(1,len(self.emissions)):
            for state in states:
                productWeights = [prevScore*self.emitMatrix[prevState][self.emissions[-i]]*self.transMatrix[state][prevState] for prevState,prevScore in graph[i-1].items()]
                graph[i][state] = sum(productWeights)

        return graph

    def forward(self):
        '''
        Runs genGraph() to crate an empty graph and then scores it from the source to the sink.
        The score of each node is equal to the sum of the weights of each edge that goes into it.
        Weight = score of previous node * transition pr * emission pr

        :return: list of dictionaries, each dictionary represents the ith layer in an HMM
                    key:value = state:score
        '''

        graph = self.genGraph()
        states = self.transMatrix.keys()

        # score first layer of graph
        for state in states:
            graph[0][state] = 1/len(states) * self.emitMatrix[state][self.emissions[0]]

        # score the rest of the graph
        for i in range(1,len(self.emissions)):
            for state in states:

                productWeights = [prevScore*self.emitMatrix[state][self.emissions[i]]*self.transMatrix[prevState][state] for prevState,prevScore in graph[i-1].items()]
                graph[i][state] = sum(productWeights)

        return graph



    def prEmissions(self):
        '''
        Calculates the probability that the HMM is at a certain state for every emission in emissions
        pr of state at time i = (score at layer i from forward graph * score at layer i from backward graph) / score of sink from forward
        :return: 2d list where hiddenMatrix[i] gives you a list of floats, at emission i, of the pr for each state in the hmm
        '''
        forwardGraph = self.forward() # creates a graph filled source to sink
        reverseGraph = self.backward() # creates a graph filled sink ot source
        sink = sum(forwardGraph[-1].values()) # stores the sink value

        hiddenMatrix = []
        hmmLength = len(self.emissions)-1 # number of layers in the HMM, assuming there is a 0 layer
        for i in range(hmmLength+1): # iterates through the HMM
            prStates = []
            for state in self.transMatrix.keys(): # calculates the pr of each state at emission i
                forward = forwardGraph[i][state]
                backward = reverseGraph[hmmLength-i][state]
                pr = (forward * backward)/sink
                prStates.append(pr)

            hiddenMatrix.append(prStates)

        return hiddenMatrix



def main(fName=''):
    '''
    Handles input/output. Generates transition matrix and path from input data and
    runs prPath to find the probability of the given path
    '''

    with open(fName) as inFile:
        lines = inFile.readlines()
        emissions = lines[0].strip()
        alphabet = lines[2].strip().split()
        states = lines[4].strip().split()

        transMatrix = {}

        for i in range(len(states)):
            row = lines[7+i].strip().split()

            rowDictionary = {}
            for j in range(len(states)):
                rowDictionary[states[j]] = float(row[j+1])
            transMatrix[row[0]] = rowDictionary

        emitMatrix = {}

        for i in range(len(states)):
            row = lines[9+len(states)+i].strip().split()

            rowDictionary = {}
            for j in range(len(alphabet)):
                rowDictionary[alphabet[j]] = float(row[j+1])
            emitMatrix[row[0]] = rowDictionary

    hmm = HMM(transMatrix,emitMatrix,emissions)

    hiddenMatrix = hmm.prEmissions()
    toPrint = ''
    for state in hmm.transMatrix.keys():
        toPrint += state + '\t'
    print(toPrint.rstrip())
    for row in hiddenMatrix:
        toPrint = ''
        for pr in row:
            toPrint += str(round(pr,4))+'\t'
        print(toPrint.rstrip())


if __name__ == '__main__':
    main('problem24in.txt')

A	B
0.5118	0.4882
0.3282	0.6718
0.2574	0.7426
0.4077	0.5923
0.4106	0.5894
0.324	0.676
0.2542	0.7458
0.3206	0.6794
0.3265	0.6735
0.4066	0.5934


Given: A string x, followed by the alphabet Σ from which x was constructed, followed by the states States, transition matrix Transition, and emission matrix Emission of an HMM (Σ, States, Transition, Emission).

Return: The probability Pr(πi = k|x) that the HMM was in state k at step i (for each state k and each step i).